In [1]:
import math
import collections
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import utils

In [2]:
MonthlyStatus = collections.namedtuple('MonthlyStatus',
                                       ['mortgage_amount', 'price_index', 'price_real_estate'])


def get_price_index(year_growth_rate, month):
    return math.pow(1 + year_growth_rate/12, month)


class Simulation:
    def __init__(self, scenario, purchase_price, mortgage, portfolio):
        self.scenario = scenario
        self.purchase_price = purchase_price
        self.mortgage = mortgage
        self.portfolio = portfolio
        self.mortgage_history = []
        self.payment_history = []
    
    def get_monthly_status(self, month):
        mortgage_amount = self.mortgage.mortgage_amount
        price_index = get_price_index(self.scenario['growth_rate_real_estate'], month)
        current_price = (self.purchase_price + (self.purchase_price * (price_index - 1)))
        return MonthlyStatus(mortgage_amount, price_index, current_price)

    def run(self):
        i = 0
        max_periods = self.mortgage.maturity * self.mortgage.n_periods
        new_monthly_payment_amount = self.mortgage.monthly_payment + self.scenario['mortgage_overpay_month']
        self.mortgage.update_monthly_payment_amount(new_monthly_payment_amount)
        while self.mortgage.mortgage_amount > 0 and i <= max_periods:
            self.mortgage_history.append(self.get_monthly_status(i))
            self.payment_history.append(self.mortgage.get_next_payment())
            current_stock_price = (get_price_index(self.scenario['growth_rate_stocks'], i)
                                   * self.scenario['stock_price'])
            self.portfolio.purchase(self.scenario['mortgage_overpay_month'], current_stock_price)
            self.portfolio.update_values(current_stock_price)
            i += 1

In [3]:
scenario_1 = {'growth_rate_real_estate': 0.05,
              'growth_rate_stocks': 0.05,
              'stock_price': 100,
              'currency_exchange_rate': 8.5,
              'mortgage_overpay_month': 5000}

port = utils.Portfolio()
mort = utils.Mortgage(50, 0.0305, 3.6e6, 30, 12)
sim = Simulation(scenario_1, 4.2e6, mort, port)

In [4]:
sim.run()

In [5]:
result = (pd.DataFrame(sim.payment_history)
          .join(pd.DataFrame(sim.mortgage_history))
          .join(pd.DataFrame.from_records([el.to_dict() for el in sim.portfolio.purchases])))

In [6]:
result['monthly_payment'] = result.interest_amount + result.capital_downpayment_amount + result.fee
result['profit_stock'] = result.value_stock - result.purchase_price_stock

In [7]:
result.head()

,fee,interest_amount,capital_downpayment_amount,mortgage_amount,price_index,price_real_estate,invested_amount,purchase_price_stock,value_stock,monthly_payment,profit_stock
0,50,9150.000000,11124.995854,3.600000e+06,1.000000,4.200000e+06,5000,100.000000,13339.483312,20324.995854,13239.483312
1,50,9121.723969,11153.271885,3.588875e+06,1.004167,4.217500e+06,5000,100.416667,13284.132759,20324.995854,13183.716092
2,50,9093.376069,11181.619785,3.577722e+06,1.008351,4.235073e+06,5000,100.835069,13229.011876,20324.995854,13128.176807
3,50,9064.956119,11210.039735,3.566540e+06,1.012552,4.252719e+06,5000,101.255216,13174.119711,20324.995854,13072.864495
4,50,9036.463935,11238.531919,3.555330e+06,1.016771,4.270439e+06,5000,101.677112,13119.455314,20324.995854,13017.778201


In [8]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 11 columns):
fee                           237 non-null int64
interest_amount               237 non-null float64
capital_downpayment_amount    237 non-null float64
mortgage_amount               237 non-null float64
price_index                   237 non-null float64
price_real_estate             237 non-null float64
invested_amount               237 non-null int64
purchase_price_stock          237 non-null float64
value_stock                   237 non-null float64
monthly_payment               237 non-null float64
profit_stock                  237 non-null float64
dtypes: float64(9), int64(2)
memory usage: 20.5 KB


In [9]:
point_in_time = 12
paid_interests = result.interest_amount.iloc[:point_in_time].sum() + result.fee.iloc[:point_in_time].sum()
profit = result.price_real_estate.iloc[point_in_time] - result.price_real_estate.iloc[0]
equity = result.capital_downpayment_amount.iloc[:point_in_time].sum()
deposit = result.price_real_estate.iloc[0] - result.mortgage_amount[0]
print('Paid interests to date: {:.0f}'.format(paid_interests))
print('Profit in case of sale: {:.0f}'.format(profit))
print('Equity to date: {:.0f}'.format(equity))

Paid interests to date: 108518
Profit in case of sale: 214880
Equity to date: 135382


In [10]:
total_revenue = equity + profit + deposit
print('Total revenue from mortgage: {:.0f}'.format(total_revenue))

Total revenue from mortgage: 950262


In [12]:
print('Total revenue from stocks: {:.0f}'.format(result.invested_amount.sum() + result.profit_stock.sum()))
print('Total profit from stocks: {:.0f}'.format(result.profit_stock.sum()))

Total revenue from stocks: 3159519
Total profit from stocks: 1974519
